<a href="https://colab.research.google.com/github/kunal-vr0/Log-Anomaly-Detection/blob/main/Anomaly_Detection_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
train_file_path = '/content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/BGL/BGL_test.txt'
data_size = 500_000 # Traning dataset size
test_file_path = '/content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/BGL_/validation.txt'
test_file_label_path = '/content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/BGL_/validation_label.csv'
save_path = '/content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
_label = []
with open('/content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/BGL/BGL.log') as logs:
  for i, log in enumerate(logs):
    if log[0] == '-':
      _label.append(0)
    else:
      _label.append(1)
    if i == data_size:
      break

import csv
with open(save_path+'/train_label.csv', 'w',newline='') as file:
  writer = csv.writer(file)
  for label in _label:
    writer.writerow([label])

In [ ]:
sum(_label)

206848

#Data Preprocessor

In [ ]:
import re
def Cleaner(text):
  pattern = [r'\d{4}-\d{2}-\d{2}-\d{2}\.\d{2}\.\d{2}\.\d{6}', r'\d{10}', r'\d{4}.\d{2}.\d{2}']
  pattern = '|'.join(pattern)
  text = re.sub(pattern, '', text).split()
  return ' '.join(text)

In [ ]:
sentence = "1123637103 2005.08.09 R34-M1-N6-C:J03-U01 2005-08-09-18.25.03.244588 R34-M1-N6-C:J03-U01 RAS KERNEL INFO CE sym 34, at 0x041cc780, mask 0x80"
Cleaner(sentence)

'R34-M1-N6-C:J03-U01 R34-M1-N6-C:J03-U01 RAS KERNEL INFO CE sym 34, at 0x041cc780, mask 0x80'

In [ ]:
import re
def Tokenizer(text):
  pattern = [r'\d{4}-\d{2}-\d{2}-\d{2}\.\d{2}\.\d{2}\.\d{6}', r'\d{10}', r'\d{4}.\d{2}.\d{2}', r':']
  pattern = '|'.join(pattern)
  text = re.sub(pattern, ' ', text)
  return text.split()

In [ ]:
sentence = "1123637103 2005.08.09 R34-M1-N6-C:J03-U01 2005-08-09-18.25.03.244588 R34-M1-N6-C:J03-U01 RAS KERNEL INFO CE sym 34, at 0x041cc780, mask 0x80"
Tokenizer(sentence)

['R34-M1-N6-C',
 'J03-U01',
 'R34-M1-N6-C',
 'J03-U01',
 'RAS',
 'KERNEL',
 'INFO',
 'CE',
 'sym',
 '34,',
 'at',
 '0x041cc780,',
 'mask',
 '0x80']

#Saving Processed Data

In [ ]:
def load_data(path):
  logs = []
  with open(path, opt='clean') as logs:
    log_list = []
    if opt == 'clean':
      for log in logs:
        log_list.append(log)
    elif opt == 'raw':
      for log in logs:
        log_list.append(Cleaner(log))
  return log

In [ ]:
log_list_train = []
with open(train_file_path) as logs:
  for i, log in enumerate(logs):
    log = Cleaner(log)
    log_list_train.append(log)
    if i == data_size:
      break
with open(save_path+'/train.txt', 'w', encoding='utf-8') as file:
  for log in log_list_train:
    file.write(log+'\n')


In [ ]:
log_list_test = []
with open(test_file_path) as logs:
  for i, log in enumerate(logs):
    log = Cleaner(log)
    log_list_test.append(log)

with open(save_path+'/test.txt', 'w', encoding='utf-8') as file:
  for log in log_list_test:
    file.write(log+'\n')

#Traning Word2Vec

In [ ]:
!pip install -q gensim
from gensim.models import Word2Vec

Create Corpus

In [ ]:
corpus = []
for log in log_list_train:
  corpus.append(Tokenizer(log))
len(corpus)

500001

In [ ]:
vector_size = 50
encoder_wv = Word2Vec(corpus, min_count=10, vector_size = vector_size)
encoder_wv.wv.key_to_index

In [ ]:
encoder_wv.save(save_path+'/word2vec.model')

#Creating Word2Vec Encoder

Writing this with the assumption in mind that i am going to use already processed data, if we have to use raw data it won't work. Will write different function for that

In [ ]:
from gensim.models import Word2Vec
import numpy as np
encoder_path = save_path+'/word2vec.model'
encoder = Word2Vec.load(encoder_path)
# text is a list of tokens ie. tokenized logs
def Encoder_wv(text, encoder, out_dim):
  word_encd = []
  for word in Tokenizer(text):
    try:
      x = encoder.wv.get_vector(word)
    except KeyError:
      x = np.zeros((out_dim,))
    word_encd.append(x)
  x = np.array(word_encd)
  x = x.mean(axis=0)
  return x

In [ ]:
test = 'R53-M1-NB-C:J13-U01 R53-M1-NB-C:J13-U01 RAS KERNEL INFO 1 ddr errors(s) detected and corrected on rank 0, symbol 25, bit 5'
Encoder_wv(test, encoder, vector_size)

array([-0.2182231 ,  0.6998633 ,  0.00782834, -1.310223  , -0.25956696,
       -0.62656164,  0.3832275 ,  0.01527553,  3.2608845 ,  0.8691459 ,
       -0.31265977,  2.055429  , -1.0796697 ,  0.6922232 , -2.1101928 ,
        0.29701012,  1.7858416 , -0.29627404, -1.306777  ,  0.55728924,
        0.05145664,  0.94345695, -0.3733741 , -2.7868617 ,  2.6080575 ,
       -2.9535592 , -1.3626403 ,  0.54713273, -1.9287565 ,  2.3886502 ,
       -1.1286439 , -1.0503771 ,  1.6921005 ,  0.96658623, -0.5869502 ,
        1.9268944 ,  0.8321146 ,  2.055301  ,  1.1270434 , -1.893717  ,
       -0.24331959, -0.9105654 ,  2.3617747 , -2.334908  , -1.2731454 ,
       -2.0654576 ,  0.53145754,  0.661648  ,  0.6731999 , -0.75321186],
      dtype=float32)

#Saving embedded log dataset

In [ ]:
import csv
from tqdm import tqdm
with open(save_path+'/train_w2v_50.csv', 'w', newline='') as file:  #change file name
  writer = csv.writer(file)
  for log in tqdm(log_list_train):
    writer.writerow(Encoder_wv(log, encoder, vector_size))

100%|██████████| 500001/500001 [00:57<00:00, 8759.70it/s] 


In [ ]:
import csv
from tqdm import tqdm
with open(save_path+'/test_w2v_50.csv', 'w', newline='') as file:  #change file name
  writer = csv.writer(file)
  for log in tqdm(log_list_test):
    writer.writerow(Encoder_wv(log, encoder, vector_size))

100%|██████████| 20000/20000 [00:02<00:00, 8875.24it/s]


#Creating Sentence-Transformer Encoder

In [ ]:
!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer

In [ ]:
encoder_tf = SentenceTransformer('all-MiniLM-L6-v2')
x = 'x may be any sentence'
encoder_tf.encode(x)

In [ ]:
import torch.nn as nn
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Encoder_tf(nn.Module):
  def __init__(self, encoder_tf, out_dim):
    super(Encoder_tf, self).__init__()
    self.encoder_tf = encoder_tf
    self.linear = nn.Linear(384, out_dim)

  def forward(self, x):
    x = self.encoder_tf.encode(x)
    x = torch.from_numpy(x).to(device)
    x = self.linear(x)
    return x

#Saving embedded log dataset

In [ ]:
out_dim = 50
Encoder_tf = Encoder_tf(encoder_tf, out_dim)
Encoder_tf.to(device)
"Model Created"

'Model Created'

In [ ]:
from tqdm import tqdm
import csv
batch_size = 512
log_encds_train = []
batch = []
with open(save_path+'/train.txt', encoding='utf-8') as logs:
  for i, log in tqdm(enumerate(logs)):
    batch.append(str(log.strip()))
    if len(batch) == batch_size or i == 0:
      batch_encds = Encoder_tf(batch)
      log_encds_train.extend(batch_encds.cpu().detach().numpy())
      batch = []
if len(batch) > 0:
  batch_encds = Encoder_tf(batch)
  log_encds_train.extend(batch_encds.cpu().detach().numpy())

print('Encodings created!!')

with open(save_path+'/train_tf_50.csv', 'w', newline='') as file:  #change file name
  writer = csv.writer(file)
  for encd in tqdm(log_encds_train):
    writer.writerow(encd)

print('Encodings saved!!')



500001it [05:37, 1480.55it/s]


Encodings created!!


100%|██████████| 500001/500001 [00:24<00:00, 20758.97it/s]

Encodings saved!!


In [ ]:
from tqdm import tqdm
import csv
batch_size = 512
log_encds_test = []
batch = []
with open(save_path+'/test.txt', encoding='utf-8') as logs:
  for i, log in tqdm(enumerate(logs)):
    batch.append(str(log.strip()))
    if len(batch) == batch_size or i == 0:
      batch_encds = Encoder_tf(batch)
      log_encds_test.extend(batch_encds.cpu().detach().numpy())
      batch = []
if len(batch) > 0:
  batch_encds = Encoder_tf(batch)
  log_encds_test.extend(batch_encds.cpu().detach().numpy())

print('Encodings created!!')

with open(save_path+'/test_tf_50.csv', 'w', newline='') as file:  #change file name
  writer = csv.writer(file)
  for encd in tqdm(log_encds_test):
    writer.writerow(encd)

print('Encodings saved!!')


20000it [00:16, 1190.71it/s]


Encodings created!!


100%|██████████| 20000/20000 [00:00<00:00, 23496.25it/s]

Encodings saved!!


Using 50 as out dim for first set of encodings. will need to save encoding for out dim of 100 later.

#BPE Tokenizer

In [ ]:
!pip install -q tokenizers

In [ ]:
log_list = []
with open(save_path+'/train.txt') as logs:
  for log in logs:
    log_list.append(str(log))

In [ ]:
from tokenizers import Tokenizer, models, trainers
tokenizer_bpe = Tokenizer(models.BPE())
trainer = trainers.BpeTrainer(vocab_size=1000, special_tokens=['<s>', '<pad>', '</s>', '<unk>'])
tokenizer_bpe.train_from_iterator(log_list, trainer)

In [ ]:
tokenizer_bpe.save(save_path+'/BPE_tokenizer.json')

In [ ]:
text = "1123637084 2005.08.09 R21-M1-NF-C:J11-U11 2005-08-09-18.24.44.452974 R21-M1-NF-C:J11-U11 RAS KERNEL INFO CE sym 9, at 0x18b2c360, mask 0x80"
tokenizer_bpe.encode(Cleaner(text)).tokens

['R2',
 '1-M1-N',
 'F',
 '-C:J11',
 '-U11 R2',
 '1-M1-N',
 'F',
 '-C:J11',
 '-U11 RAS KERNEL INFO CE sym ',
 '9',
 ', at 0x',
 '18',
 'b',
 '2',
 'c',
 '3',
 '6',
 '0, mask 0x',
 '80']

Unable to load saved BPE tokenizer

#Traning W2V with BPE

In [ ]:
!pip install -q gensim
from gensim.models import Word2Vec

create corpus

In [ ]:
corpus = []
log_list_train = []
with open(save_path+'/train.txt') as logs:
  for log in logs:
    log_list_train.append(log)
    corpus.append(tokenizer_bpe.encode(log).tokens)
len(corpus)

500001

In [ ]:
vector_size = 50
encoder_bpe = Word2Vec(corpus, min_count=10, vector_size = vector_size)
encoder_bpe.wv.key_to_index

{'3': 0,
 '4': 1,
 '1': 2,
 '2': 3,
 '7': 4,
 '6': 5,
 '5': 6,
 ' ': 7,
 '0': 8,
 '9': 9,
 'e': 10,
 '8': 11,
 'C': 12,
 'A': 13,
 'a': 14,
 'E': 15,
 'd': 16,
 'NULL NULL RAS MMCS ERROR idoproxydb hit ASSERT condition: ASSERT expression=0 Source file=idotransportmgr.cpp Source line=1043 Function=int IdoTransportMgr::SendPacket(IdoUdpMgr*, BglCtlPavTrace*)\n': 17,
 'c': 18,
 'B': 19,
 '.': 20,
 '\n': 21,
 '/': 22,
 'p': 23,
 '_': 24,
 '-U11 RAS KERNEL INFO ': 25,
 'D': 26,
 'F': 27,
 ':': 28,
 'T': 29,
 '-U01 RAS KERNEL INFO ': 30,
 'R6': 31,
 'f': 32,
 '00': 33,
 'e ': 34,
 '5-M0-N': 35,
 'c\n': 36,
 's': 37,
 't': 38,
 'o': 39,
 '1-M1-N': 40,
 '10': 41,
 '4-M1-N': 42,
 'w': 43,
 'b': 44,
 '-C:J02': 45,
 '5-M1-N': 46,
 '1-M0-N': 47,
 'u': 48,
 'm': 49,
 '-C:J07': 50,
 '3-M1-N': 51,
 'er': 52,
 '-C:J06': 53,
 'R0': 54,
 ', at 0x0': 55,
 '2-M1-N': 56,
 'L': 57,
 '-C:J09': 58,
 '3-M0-N': 59,
 '-C:J08': 60,
 '-C:J15': 61,
 '0-M0-N': 62,
 'R': 63,
 'l': 64,
 '-C:J17': 65,
 'R2': 66,
 'R1':

In [ ]:
encoder_bpe.save(save_path+'/bpe_word2vec.model')

#W2V with BPE tokenizer

In [ ]:
from gensim.models import Word2Vec
import numpy as np
#encoder_path = save_path+'/word2vec.model'
#encoder = Word2Vec.load(encoder_path)
# text is a single clean log

def Encoder_bpe(text, out_dim = 50):
  text = tokenizer_bpe.encode(text).tokens
  word_encd = []
  for word in text:
    try:
      x = encoder_bpe.wv.get_vector(word)
    except KeyError:
      x = np.zeros((out_dim,))
    word_encd.append(x)
  x = np.array(word_encd)
  x = x.mean(axis=0)
  return x

In [ ]:
test = 'R53-M1-NB-C:J13-U01 R53-M1-NB-C:J13-U01 RAS KERNEL INFO 1 ddr errors(s) detected and corrected on rank 0, symbol 25, bit 5'
Encoder_bpe(Cleaner(test))

array([ 0.75907815, -1.04062   ,  1.0004829 , -1.8830768 , -1.2168101 ,
        0.96167123,  0.82348686, -3.0034652 , -1.7424    , -3.735474  ,
       -0.64232343,  1.068114  , -0.8800181 , -2.0468874 , -1.4233224 ,
        0.20631637, -1.1317705 , -0.1316223 , -0.99989676,  0.22540484,
       -0.32084846,  0.48956272, -0.65741163,  0.6877553 ,  0.5431522 ,
        0.46263033,  2.4752517 ,  0.06142056,  1.4557103 ,  1.1757616 ,
        1.6000913 ,  1.2547634 ,  0.01456471, -1.7094446 ,  0.4439111 ,
        0.9882942 , -1.8566906 , -0.44048327,  1.9332047 , -0.29369247,
        1.6866869 , -0.77969337,  0.14128543,  1.3626131 ,  1.3455446 ,
        1.1051676 ,  0.51945263, -4.2469764 , -1.9163488 ,  0.15183213],
      dtype=float32)

#Saving embedded dataset

In [ ]:
import csv
from tqdm import tqdm
with open(save_path+'/train_bpe_50.csv', 'w', newline='') as file:  #change file name
  writer = csv.writer(file)
  for log in tqdm(log_list_train):
    writer.writerow(Encoder_bpe(log))

100%|██████████| 500001/500001 [01:02<00:00, 7954.60it/s] 


In [ ]:
log_list_test = []
with open(save_path+'/test.txt') as logs:
  for log in logs:
    log_list_test.append(log)

In [ ]:
import csv
from tqdm import tqdm
with open(save_path+'/test_bpe_50.csv', 'w', newline='') as file:  #change file name
  writer = csv.writer(file)
  for log in tqdm(log_list_test):
    writer.writerow(Encoder_bpe(log))

100%|██████████| 20000/20000 [00:02<00:00, 9083.11it/s]


#Traning Anomaly Transformer

In [ ]:
#dataset = 'BGL_'
data_path = '/content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL'
input_c = out_dim
output_c = out_dim

Remember to change
1. Train and test file names
2. checkpoint name
3. Input size (out_dim)

#1. W2V, dim = 50

In [ ]:
%cd /content/drive/MyDrive/LLM_cx/Anomaly-Transformer

/content/drive/MyDrive/LLM_cx/Anomaly-Transformer


In [ ]:
!export CUDA_VISIBLE_DEVICES=0

!python main.py --anormly_ratio 1 --num_epochs 3 --batch_size 256 --mode train --dataset BGL_ --data_path /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL --input_c 50 --output_c 50
!python main.py --anormly_ratio 1 --num_epochs 10 --batch_size 256 --mode test --dataset BGL_ --data_path /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL --input_c 50 --output_c 50 --pretrained_model 20

------------ Options -------------
anormly_ratio: 1.0
batch_size: 256
data_path: /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL
dataset: BGL_
input_c: 50
k: 3
lr: 0.0001
mode: train
model_save_path: checkpoints
num_epochs: 3
output_c: 50
pretrained_model: None
win_size: 100
-------------- End ----------------
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
======================TRAIN MODE======================
	speed: 0.6774s/iter; left time: 3901.6323s
	speed: 0.6788s/iter; left time: 3841.8563s
	speed: 0.6863s/iter; left time: 3816.0138s
	speed: 0.6896s/iter; left time: 3765.3675s
	speed: 0.6923s/iter; left time: 3710.9220s
	speed: 0.6929s/iter; left time: 3644.8294s
	speed: 0.6921s/iter; left time: 3571.4118s
	speed: 0.6923s/iter; left time: 3503.2076s
	speed: 0.6911s/iter

In [ ]:
!python main.py --anormly_ratio 1 --num_epochs 10 --batch_size 256 --mode test --dataset BGL_ --data_path /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL --input_c 50 --output_c 50 --pretrained_model 20

------------ Options -------------
anormly_ratio: 1.0
batch_size: 256
data_path: /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL
dataset: BGL_
input_c: 50
k: 3
lr: 0.0001
mode: test
model_save_path: checkpoints
num_epochs: 10
output_c: 50
pretrained_model: 20
win_size: 100
-------------- End ----------------
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
======================TEST MODE======================
/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
Threshold : 0.0014044168835971463
pred:    (19900,)
gt:      (19900,)
pred:  (19900,)
gt:    (19900,)
TP: 285, FP: 230, TN: 19358, FN: 27
285 230 19358 27
515
312
Accuracy 

#2. S_TF, dim = 50

In [ ]:
!export CUDA_VISIBLE_DEVICES=0

!python main.py --anormly_ratio 1 --num_epochs 3 --batch_size 256 --mode train --dataset BGL_ --data_path /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL --input_c 50 --output_c 50
!python main.py --anormly_ratio 1 --num_epochs 10 --batch_size 256 --mode test --dataset BGL_ --data_path /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL --input_c 50 --output_c 50 --pretrained_model 20

------------ Options -------------
anormly_ratio: 1.0
batch_size: 256
data_path: /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL
dataset: BGL_
input_c: 50
k: 3
lr: 0.0001
mode: train
model_save_path: checkpoints
num_epochs: 3
output_c: 50
pretrained_model: None
win_size: 100
-------------- End ----------------
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
======================TRAIN MODE======================
	speed: 0.6751s/iter; left time: 3888.4700s
	speed: 0.6862s/iter; left time: 3883.8802s
	speed: 0.6917s/iter; left time: 3846.0649s
	speed: 0.6957s/iter; left time: 3798.6741s
	speed: 0.6982s/iter; left time: 3742.1300s
	speed: 0.6990s/iter; left time: 3676.5404s
	speed: 0.6994s/iter; left time: 3609.1335s
	speed: 0.6982s/iter; left time: 3532.9841s
	speed: 0.6985s/iter

#3. BPE W2V, dim = 50

In [ ]:
%cd /content/drive/MyDrive/Anomaly-Transformer

/content/drive/.shortcut-targets-by-id/1a1eKLOfAoKgnjEEnc0zFX_lA4weTP_gV/Anomaly-Transformer


training this from different account, directories would change

In [ ]:
!export CUDA_VISIBLE_DEVICES=0

!python main.py --anormly_ratio 1 --num_epochs 3 --batch_size 256 --mode train --dataset BGL_ --data_path /content/drive/MyDrive/Anomaly-Transformer/dataset/My_BGL --input_c 50 --output_c 50
!python main.py --anormly_ratio 1 --num_epochs 10 --batch_size 256 --mode test --dataset BGL_ --data_path /content/drive/MyDrive/Anomaly-Transformer/dataset/My_BGL --input_c 50 --output_c 50 --pretrained_model 20

------------ Options -------------
anormly_ratio: 1.0
batch_size: 256
data_path: /content/drive/MyDrive/Anomaly-Transformer/dataset/My_BGL
dataset: BGL_
input_c: 50
k: 3
lr: 0.0001
mode: train
model_save_path: checkpoints
num_epochs: 3
output_c: 50
pretrained_model: None
win_size: 100
-------------- End ----------------
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
======================TRAIN MODE======================
	speed: 0.6837s/iter; left time: 3938.2967s
	speed: 0.6935s/iter; left time: 3925.3569s
	speed: 0.6905s/iter; left time: 3839.0664s
	speed: 0.6906s/iter; left time: 3770.4057s
	speed: 0.6900s/iter; left time: 3698.4958s
	speed: 0.6891s/iter; left time: 3624.7958s
	speed: 0.6888s/iter; left time: 3554.1231s
	speed: 0.6887s/iter; left time: 3484.6835s
	speed: 0.6883s/iter; left 

#Testing with epochs

In [ ]:
%cd /content/drive/MyDrive/LLM_cx/Anomaly-Transformer

/content/drive/MyDrive/LLM_cx/Anomaly-Transformer


In [ ]:
!export CUDA_VISIBLE_DEVICES=0

!python main.py --anormly_ratio 1 --num_epochs 5 --batch_size 256 --mode train --dataset BGL_ --data_path /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL --input_c 50 --output_c 50
!python main.py --anormly_ratio 1 --num_epochs 10 --batch_size 256 --mode test --dataset BGL_ --data_path /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL --input_c 50 --output_c 50 --pretrained_model 20

------------ Options -------------
anormly_ratio: 1.0
batch_size: 256
data_path: /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL
dataset: BGL_
input_c: 50
k: 3
lr: 0.0001
mode: train
model_save_path: checkpoints
num_epochs: 5
output_c: 50
pretrained_model: None
win_size: 100
-------------- End ----------------
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
test: (19999, 50)
train: (500000, 50)
test_label: (19999, 1)
======================TRAIN MODE======================
	speed: 0.7068s/iter; left time: 6832.2850s
	speed: 0.7384s/iter; left time: 7063.3581s
	speed: 0.7345s/iter; left time: 6952.7986s
	speed: 0.7305s/iter; left time: 6842.2589s
	speed: 0.7311s/iter; left time: 6774.3135s
	speed: 0.7297s/iter; left time: 6688.2184s
	speed: 0.7296s/iter; left time: 6614.3060s
	speed: 0.7293s/iter; left time: 6539.2767s
	speed: 0.7284s/iter

In [ ]:
%cd /content/drive/MyDrive/LLM_cx/Anomaly-Transformer


/content/drive/MyDrive/LLM_cx/Anomaly-Transformer


In [ ]:
!python main.py --anormly_ratio 1 --num_epochs 10 --batch_size 256 --mode test --dataset BGL_ --data_path /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL --input_c 50 --output_c 50 --pretrained_model 20

------------ Options -------------
anormly_ratio: 1.0
batch_size: 256
data_path: /content/drive/MyDrive/LLM_cx/Anomaly-Transformer/dataset/My_BGL
dataset: BGL_
input_c: 50
k: 3
lr: 0.0001
mode: test
model_save_path: checkpoints
num_epochs: 10
output_c: 50
pretrained_model: 20
win_size: 100
-------------- End ----------------
test: (500000, 50)
train: (500000, 50)
test_label: (500000, 1)
test: (500000, 50)
train: (500000, 50)
test_label: (500000, 1)
test: (500000, 50)
train: (500000, 50)
test_label: (500000, 1)
test: (500000, 50)
train: (500000, 50)
test_label: (500000, 1)
======================TEST MODE======================
/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
Threshold : 0.004331833287142203
pred:    (500000,)
gt:      (500000,)
pred:  (500000,)
gt:    (500000,)
TP: 205819, FP: 2993, TN: 290159, FN: 1029
205819 2993 2901